In [ ]:
import os
import time
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from  google.cloud import bigquery

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'c:/Users/Diego/Desktop/sql-practice-cred.json'

In [ ]:
client = bigquery.Client()

In [ ]:
sns.set_style('dark')
sns.set_palette('mako')

In [ ]:
def query_result(query):
    query_job = client.query(query)

    while query_job.state != 'DONE':
        query_job.reload()
        time.sleep(3)

    if query_job.state == 'DONE':
        df = query_job.to_dataframe()
        return df
    else:
        print(query_job.result())

In [ ]:
query = """
WITH weekdays AS(
SELECT ['Sun', 'Mon', 'Tues', 'Wed', 'Thurs', 'Fri', 'Sat'] AS daysofweek)

SELECT
  ROUND(AVG(trip_total), 2) trip_total_cost,
  ROUND(AVG(tips), 2) avg_tips,
  CONCAT(ROUND(AVG(trip_seconds) / 60, 2), ' hrs') trip_duration,
  daysofweek[ORDINAL(EXTRACT(DAYOFWEEK FROM trip_start_timestamp))] weekday,
  EXTRACT(HOUR FROM trip_start_timestamp) hour
FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`, weekdays
WHERE trip_seconds > 0
  AND fare > 0
GROUP BY 4, 5
ORDER BY 4, 5;
"""

trip_stats_weekdayhour = query_result(query)

trip_stats_weekdayhour.head()

In [ ]:
sns.relplot(
    data=trip_stats_weekdayhour,
    x='trip_total_cost',
    y='avg_tips',
    hue='weekday',
        hue_order=['Mon','Tue','Wed','Thurs','Fri','Sat','Sun'],
    palette=sns.color_palette('rocket', 7),
    kind='scatter'
)

In [ ]:
query = """

"""